In [1]:
import math
import numpy as np

- wycena - funkcja generująca wycenę racjonalną na podstawie wzoru rekurencyjnego z polecenia zadania
- generuj_ceny - na podstawie *S_0*, *u* i *d* tworzy drzewo z możliwymi cenami
- zwroc_wartosc - podając drzewo wycen i omegę w postaci listy z wartościami binarnymi (gdzie 1 odpowiada *u* a 0 odpowiada *d*, np. [1,0,0,1,0] to uddud) otrzymujemy wartość dla konkretnej omegi 

In [2]:
class CRR:
    def __init__(self, d, u, r):
        self.d = d
        self.u = u
        self.r = r
    
    def wycena (self,X):
        T = int(math.log(len(X),2))
        p_mart = (1+self.r-self.d)/(self.u-self.d)
        cena_r = [None for i in range (T+1)]
        cena_r[T] = X
        for p in range(T-1,-1,-1):
            cena_r[p] = [(p_mart*cena_r[p+1][i+1]+(1-p_mart)*cena_r[p+1][i])/(1+self.r) for i in range(0,len(cena_r[p+1])-1,2)]
        return cena_r
    
    def generuj_ceny (self,t, S_0):
        S = [None for i in range(t+1)]
        S[0] = [S_0]
        for i in range(1,t+1):
            S[i] = [f(s) for s in S[i-1] for f in (lambda x: x*self.d, lambda x: x*self.u)]
        return S
    
    def zwroc_wartosc(self,tree,omega):
        k = sum([bit * 2**n for n,bit in enumerate(reversed(omega))])
        t = len(omega)
        return(tree[t][k])
    

# Pdpkt B)

Funkcja implementująca wzór z zadania, jako argumenty przyjmuje T i tworzy listę dla kolejnych t, rynek, strike, oraz informację, czy wyceniamy opcję *call* czy *put* 

In [3]:
def Teoretyczne(T, crr,S_0, K, opcja):
    if opcja == "call":
        flag = 1
    else:
        flag = -1
    p_mart = (1+crr.r-crr.d)/(crr.u-crr.d)
    val = np.array([None for i in range(T+1)])
    S = crr.generuj_ceny(T, S_0)
    
    for t in range (T+1):
        s = np.array(S[T-t])
        val[T-t] = sum([math.comb(t,j)*p_mart**j*(1-p_mart)**(t-j)*np.maximum(flag*(s*crr.u**j*crr.d**(t-j)-K),0) for j in range(t+1)])/(1+crr.r)**t  
    
    return val

In [4]:
S_0 = 100
u = 1.3
d = 0.8
r = 0.1
T = 10
K = 90

## Opcja call

In [5]:
crr = CRR(d, u, r)
S = crr.generuj_ceny(T, S_0)
C = [np.maximum(np.array(s) - K,0) for s in S]

In [6]:
wycena_model = crr.wycena(C[-1])
wycena_teoretyczna = Teoretyczne(10,crr,S_0,90, "call")

In [7]:
sum([sum(abs(a-b)) for a,b in zip(wycena_model,wycena_teoretyczna)])

5.244027434514464e-12

Widzimy, że różnica w wycenach wynika jedynie z błędów obliczeń.

## Opcja call

In [8]:
crr = CRR(d, u, r)
S = crr.generuj_ceny(T, S_0)
C = [np.maximum(K - np.array(s),0) for s in S]

In [9]:
wycena_model = crr.wycena(C[-1])
wycena_teoretyczna = Teoretyczne(10,crr,S_0, 90, "put")

In [10]:
sum([sum(abs(a-b)) for a,b in zip(wycena_model,wycena_teoretyczna)])

7.910547217271358e-13

Ponownie, wszystko się zgadza.